<a href="https://colab.research.google.com/github/SofiaRguez1020/Topicos_Industria_I/blob/main/TI_1_Practica4_Sofia_Rodriguez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 4

**Nombre:** Sofia Daniela Rodriguez Saenz

**e-mail:** sofia.rodriguez5540@alumnos.udg.mx

# MODULES

In [1]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import cauchy
from scipy.stats import levy_stable

import panel as pn
import panel.widgets as pnw

pn.extension('plotly')

# CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# FUNCTIONS

In [3]:
###############################################################################################
# Brownian Motion
# This function generates a BM walk
###############################################################################################
def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
  """
  Arguments:
    n_steps: number of steps the Brownian Trajectory will take -> int
    speed: speed of the trajectory or step size -> int
    s_pos: initial position -> [x,y] list
  Returns:
    BM_2d_df: DataFrame with x,y points of the full trajectory
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

In [4]:
###############################################################################################
# Correlated Random Walk
# This function generates a CRW Walk
###############################################################################################
def crw(n_steps=1000, speed=5, s_pos=[0,0], CRW_exponent = 0.5):
  """
  Arguments:
    n_steps: number of steps the Brownian Trajectory will take -> int
    speed: speed of the trajectory or step size -> int
    s_pos: initial position -> [x,y] list
  Returns:
    CRW_df: DataFrame with x,y points of the full trajectory
  """
  #Init DFs
  CRW_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
  CRW_df = pd.concat([CRW_df, temp_df], ignore_index=True)

  # Init velocity vector
  velocity = Vec2d(speed,0)

  for i in range(n_steps-1):
    # Select turn angle
    turn_angle = wrapcauchy.rvs(c=CRW_exponent)
    #
    velocity = velocity.rotated(turn_angle)
    #Update position
    temp_df = pd.DataFrame([{'x_pos': CRW_df.x_pos[i]+velocity.x, 'y_pos': CRW_df.y_pos[i]+velocity.y}])
    CRW_df = pd.concat ([CRW_df,temp_df], ignore_index=True)
  return CRW_df

In [5]:
###############################################################################################
# Levy flight
# This function generates a levy flight
###############################################################################################
def levy_flight(n_steps=1000, init_speed = 5, s_pos=[0,0], CRW_exponent = 0.5, alpha = 1.5, beta = 0, loc=3.0):
  #Init DFs
  Levy_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
  Levy_df = pd.concat([Levy_df, temp_df], ignore_index=True)

  # Init velocity vector
  velocity = Vec2d(init_speed,0)

  for i in range(n_steps-1):
    # Select turn angle
    turn_angle = wrapcauchy.rvs(c=CRW_exponent)
    #step size
    step_size = levy_stable.rvs(alpha=alpha, beta=beta, loc=loc)

    velocity = velocity.rotated(turn_angle)
    #Update position
    temp_df = pd.DataFrame([{'x_pos': Levy_df.x_pos[i]+(velocity.x*step_size), 'y_pos': Levy_df.y_pos[i]+(velocity.y*step_size)}])
    Levy_df = pd.concat ([Levy_df,temp_df], ignore_index=True)
  return Levy_df

In [6]:
#######################################################################################################
# Euclidean distance
# This function estimates euclidean distance between two points
#######################################################################################################
def euclidean_distance(q, p):
  a2 = (q[0] - p[0])**2
  b2 = (q[1] - p[1])**2
  c = math.sqrt(a2+b2)
  return c

#######################################################################################################
# Path length calculation
# This function estimates the total path length based on euclidean distance between consecutive steps
#######################################################################################################
def path_length(trajectory):
  """
  Arguments:
    trajectory: Data set containing complete trajectory
  Returns:
    distance: calculated euclidean distance between 2 points
  """
  dis_aux = []
  for i in range (1,trajectory.shape[0]):
    dis_aux.append(euclidean_distance(trajectory.iloc[i-1], trajectory.iloc[i]))
  return pd.DataFrame(np.cumsum(dis_aux), columns=['PL'])

In [59]:
#######################################################################################################
# Mean Squared Displacement
# This function estimates the MSD
#######################################################################################################
def msd(trajectory, n=50):
  """
  Arguments:
    trajectory: Data set containing complete trajectory
    n: Window size
  Returns:
    msd_df: calculated mean squared
  """
  displacement_vec = np.array([euclidean_distance(trajectory.iloc[i-n], trajectory.iloc[i])**2 for i in range(n, trajectory.shape[0],1)])
  msd_df = pd.DataFrame(np.cumsum(displacement_vec)/(trajectory.shape[0] - n), columns=['MSD'])
  return msd_df

#Dashboard implementation

In [74]:
pn.extension()
#Define DF to track trajectory DF for both trajectoty plot and metric plot
global traj_global_df
traj_global_df = None

#Define widget layout
trajectory_type = pn.widgets.ToggleGroup(name='Panel', options=['BM', 'CRW', 'LF'], behavior="radio", width = 320, align='center')
n_steps = pn.widgets.IntSlider(name='Number of steps:', start=10, end=1000, value=500, width = 320, align = 'center')
x_pos = pn.widgets.IntInput(name='xInitPos', value=0, width = 150)
y_pos = pn.widgets.IntInput(name='yInitPos', value=0, width = 150)
speed = pn.widgets.IntSlider(name='Speed', start=1, end=15, value=5, width = 320, align='center')
metric = pn.widgets.Select(name='Metric', options=['PL','MSD'], width=320,align='center')
crw_exponent = pn.widgets.FloatInput(name='Cauchy coefficient', width=320, visible=False, align='center', step=1e-1, start=1e-1, end=1, value = 0.5)
alpha = pn.widgets.FloatInput(name='Alpha', visible=False, width=320, align='center', step=1e-1, start=1e-1, end=2, value = 1.5)

#Dinamically show or hide option for cauchy coef/levy exp
@pn.depends(trajectory_type, watch=True)
def show_input(trajectory_type):
    #Dinamically show or hide CRW and Alpha inputs
    if trajectory_type == 'BM':
        crw_exponent.visible = False
        alpha.visible = False
    elif trajectory_type == 'CRW':
        crw_exponent.visible = True
        alpha.visible = False
    elif trajectory_type == 'LF':
        crw_exponent.visible = True
        alpha.visible = True

#Define function to draw trajectory

@pn.depends(n_steps, x_pos, y_pos, speed, trajectory_type, crw_exponent, alpha)
def plot_traj(n_steps, x_pos, y_pos, speed,trajectory_type, crw_exponent, alpha):

  #Call trajectory function according to selection
  if trajectory_type == 'BM':
    traj_df = bm_2d(n_steps, speed, [x_pos, y_pos])
  elif trajectory_type == 'CRW':
    traj_df = crw(n_steps, speed, [x_pos, y_pos], crw_exponent)
  elif trajectory_type == 'LF':
    traj_df = levy_flight(n_steps, speed, [x_pos, y_pos], crw_exponent, alpha)

  #Save trajectory to aux var
  global traj_global_df
  traj_global_df = traj_df

  #Graph trajectory
  fig_traj_rw = go.Figure()
  fig_traj_rw.add_trace(go.Scatter3d(
                        x = traj_df.x_pos,
                        y = traj_df.y_pos,
                        z = traj_df.index,
                        name = trajectory_type,
                        mode = 'lines',
                        showlegend = True
                      ))

  #Add axis names and dinamically changes plot title
  fig_traj_rw.update_layout(
                    title = trajectory_type + ' Trajectory in 3D',
                    scene=dict(
                    xaxis_title='x_pos',
                    yaxis_title='y_pos',
                    zaxis_title='Time'))
  return fig_traj_rw


#Define function to draw metric plot

@pn.depends(n_steps, x_pos, y_pos, speed, trajectory_type, crw_exponent, alpha, metric)
def plot_metric(n_steps, x_pos, y_pos, speed, trajectory_type, crw_exponent, alpha, metric):
  #Get trajectory
  global traj_global_df
  if traj_global_df is None:
        return None
  if metric == 'MSD':
    metric_df = msd(traj_global_df, int(n_steps/2))
  elif metric == 'PL':
    metric_df = path_length(traj_global_df)
  #Graph trajectory
  fig_metric_rw = go.Figure()
  fig_metric_rw.add_trace(go.Scatter(
                        x = metric_df.index,
                        y = metric_df[metric],
                        name = trajectory_type,
                        mode = 'lines',
                        showlegend = True
                      ))

  #Add axis names and dinamically changes plot title
  fig_metric_rw.update_layout(
                    title = metric,
                    scene=dict(
                    xaxis_title='x_pos',
                    yaxis_title='y_pos',
                    zaxis_title='Time'))
  return fig_metric_rw


In [79]:
dashboard = pn.Row(
    pn.WidgetBox(pn.pane.Markdown("# Panel options", align='center'),
                 trajectory_type,
                 n_steps,
                 pn.Row(x_pos, y_pos, align = 'center'),
                 speed,
                 metric,
                 crw_exponent,
                 alpha,
                 width= 380,
                 height = 500, align = 'center'),
    pn.Column(plot_traj, width = 500,height = 500, align='center'),
    pn.Column(plot_metric, width = 500,height = 500, align='center')
)


In [84]:
#Show dashboard embedded in colan
dashboard.servable()


Row
    [0] WidgetBox(align='center', height=500, sizing_mode='fixed', width=380)
        [0] Markdown(str, align='center')
        [1] RadioButtonGroup(align='center', name='Panel', options=['BM', 'CRW', 'LF'], value='BM', width=320)
        [2] IntSlider(align='center', end=1000, name='Number of steps:', start=10, value=500, width=320)
        [3] Row(align='center')
            [0] IntInput(name='xInitPos', width=150)
            [1] IntInput(name='yInitPos', width=150)
        [4] IntSlider(align='center', end=15, name='Speed', start=1, value=5, width=320)
        [5] Select(align='center', name='Metric', options=['PL', 'MSD'], value='MSD', width=320)
        [6] FloatInput(align='center', end=1, name='Cauchy coefficient', start=0.1, value=0.5, visible=False, width=320)
        [7] FloatInput(align='center', end=2, name='Alpha', start=0.1, value=1.2999999999999998, visible=False, width=320)
    [1] Column(align='center', height=500, sizing_mode='fixed', width=500)
        [0] ParamFunction(function, _pane=Plotly, defer_load=False)
    [2] Column(align='center', height=500, sizing_mode='fixed', width=500)
        [0] ParamFunction(function, _pane=Plotly, defer_load=False)

In [85]:
#Show dashboard in localhost
dashboard.show()

Launching server at http://localhost:35073


## Notas

Link a codigo python unicamente se encuentra aqui:
https://github.com/SofiaRguez1020/Topicos_Industria_I/blob/main/ti_1_practica4_sofia_rodriguez.py